<a href="https://colab.research.google.com/github/PalomaMuniz777/LiterAlura/blob/main/aluraColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import com.fasterxml.jackson.annotation.JsonIgnoreProperties;
import com.fasterxml.jackson.annotation.JsonAlias;
import com.fasterxml.jackson.databind.ObjectMapper;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.boot.CommandLineRunner;
import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.data.jpa.repository.JpaRepository;
import org.springframework.stereotype.Component;
import org.springframework.stereotype.Service;

import javax.persistence.*;
import java.io.IOException;
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.List;
import java.util.Optional;
import java.util.Scanner;

@SpringBootApplication
public class LiterAluraApplication {

    public static void main(String[] args) {
        SpringApplication.run(LiterAluraApplication.class, args);
    }
}

@Component
class LiterAluraCommandLineRunner implements CommandLineRunner {

    @Autowired
    private LivroService livroService;

    @Override
    public void run(String... args) throws Exception {
        Scanner scanner = new Scanner(System.in);

        while (true) {
            System.out.println("\nLiterAlura - Catálogo de Livros");
            System.out.println("1. Buscar livro pelo título");
            System.out.println("2. Listar livros registrados");
            System.out.println("3. Listar nossos autores");
            System.out.println("4. Listar autores em determinado ano");
            System.out.println("5. Listar livros em determinado idioma");
            System.out.println("0. Sair");
            System.out.print("Selecione uma opção: ");

            int opcao = scanner.nextInt();
            scanner.nextLine(); // Consumir a quebra de linha

            switch (opcao) {
                case 1:
                    System.out.print("Digite o título do livro: ");
                    String titulo = scanner.nextLine();
                    livroService.buscarLivroPorTitulo(titulo);
                    break;
                case 2:
                    livroService.listarLivros();
                    break;
                case 3:
                    livroService.listarAutores();
                    break;
                case 4:
                    System.out.print("Digite o ano: ");
                    int ano = scanner.nextInt();
                    livroService.listarAutoresVivosEmAno(ano);
                    break;
                case 5:
                    System.out.print("Digite o idioma (PT, ES, FR, EN): ");
                    String idioma = scanner.nextLine();
                    livroService.listarLivrosPorIdioma(idioma);
                    break;
                case 0:
                    System.out.println("Saindo do LiterAlura...");
                    scanner.close();
                    System.exit(0);
                default:
                    System.out.println("Opção inválida. Por favor, selecione uma opção válida.");
            }
        }
    }
}

@Service
class LivroService {

    @Autowired
    private LivroRepository livroRepository;
    @Autowired
    private AutorRepository autorRepository;

    public void buscarLivroPorTitulo(String titulo) throws IOException, InterruptedException {
        String url = "https://gutendex.com/books/?title=" + titulo;
        HttpClient client = HttpClient.newHttpClient();
        HttpRequest request = HttpRequest.newBuilder(URI.create(url))
                .build();
        HttpResponse<String> response = client.send(request, HttpResponse.BodyHandlers.ofString());

        if (response.statusCode() == 200) {
            ObjectMapper mapper = new ObjectMapper();
            List<Livro> livros = mapper.readValue(response.body(), mapper.getTypeFactory().constructCollectionType(List.class, Livro.class));

            if (!livros.isEmpty()) {
                Livro livro = livros.get(0);
                Autor autor = new Autor();
                if (!livro.getAutores().isEmpty()) {
                    autor = livro.getAutores().get(0);
                    autorRepository.save(autor);
                }
                livro.setAutor(autor);
                livroRepository.save(livro);
                System.out.println("Livro adicionado com sucesso!");
            } else {
                System.out.println("Nenhum livro encontrado com esse título.");
            }
        } else {
            System.out.println("Erro ao consultar a API: " + response.statusCode());
        }
    }

    public void listarLivros() {
        List<Livro> livros = livroRepository.findAll();

        if (!livros.isEmpty()) {
            System.out.println("\nLivros Registrados:");
            livros.forEach(livro -> System.out.println(livro));
        } else {
            System.out.println("\nNenhum livro registrado no banco de dados.");
        }
    }

    public void listarAutores() {
        List<Autor> autores = autorRepository.findAll();

        if (!autores.isEmpty()) {
            System.out.println("\nAutores Registrados:");
            autores.forEach(autor -> System.out.println(autor));
        } else {
            System.out.println("\nNenhum autor registrado no banco de dados.");
        }
    }

    public void listarAutoresVivosEmAno(int ano) {
        List<Autor> autores = autorRepository.findByAnoFalecimentoGreaterThanEqual(ano);

        if (!autores.isEmpty()) {
            System.out.println("\nAutores Vivos em " + ano + ":");
            autores.forEach(autor -> System.out.println(autor));
        } else {
            System.out.println("\nNenhum autor encontrado vivo em " + ano + ".");
        }
    }

    public void listarLivrosPorIdioma(String idioma) {
        List<Livro> livros = livroRepository.findByIdioma(idioma);

        if (!livros.isEmpty()) {
            System.out.println("\nLivros em " + idioma + ":");
            livros.forEach(livro -> System.out.println(livro));
        } else {
            System.out.println("\nNenhum livro encontrado em " + idioma + ".");
        }
    }
}

@Entity
class Livro {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    private String titulo;

    @ManyToOne
    private Autor autor;

    private String idioma;

    private Integer downloads;

    @JsonIgnoreProperties("livros")
    @JsonAlias("authors")
    private List<Autor> autores;

    public Livro() {
    }

    public Livro(String titulo, Autor autor, String idioma, Integer downloads, List<Autor> autores) {
        this.titulo = titulo;
        this.autor = autor;
        this.idioma = idioma;
        this.downloads = downloads;
        this.autores = autores;
    }

    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public String getTitulo() {
        return titulo;
    }

    public void setTitulo(String titulo) {
        this.titulo = titulo;
    }

    public Autor getAutor() {
        return autor;
    }

    public void setAutor(Autor autor) {
        this.autor = autor;
    }

    public String getIdioma() {
        return idioma;
    }

    public void setIdioma(String idioma) {
        this.idioma = idioma;
    }

    public Integer getDownloads() {
        return downloads;
    }

    public void setDownloads(Integer downloads) {
        this.downloads = downloads;
    }

    public List<Autor> getAutores() {
        return autores;
    }

    public void setAutores(List<Autor> autores) {
        this.autores = autores;
    }

    @Override
    public String toString() {
        return "Livro{" +
                "id=" + id +
                ", titulo='" + titulo + '\'' +
                ", autor=" + autor +
                ", idioma='" + idioma + '\'' +
                ", downloads=" + downloads +
                '}';
    }
}

@Entity
class Autor {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    private String nome;

    private Integer anoNascimento;

    private Integer anoFalecimento;

    public Autor() {
    }

    public Autor(String nome, Integer anoNascimento, Integer anoFalecimento) {
        this.nome = nome;
        this.anoNascimento = anoNascimento;
        this.anoFalecimento = anoFalecimento;
    }

    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public String getNome() {
        return nome;
    }

    public void setNome(String nome) {
        this.nome = nome;
    }

    public Integer getAnoNascimento() {
        return anoNascimento;
    }

    public void setAnoNascimento(Integer anoNascimento) {
        this.anoNascimento = anoNascimento;
    }

    public Integer getAnoFalecimento() {
        return anoFalecimento;
    }

    public void setAnoFalecimento(Integer anoFalecimento) {
        this.anoFalecimento = anoFalecimento;
    }

    @Override
    public String toString() {
        return "Autor{" +
                "id=" + id +
                ", nome='" + nome + '\'' +
                ", anoNascimento=" + anoNascimento +
                ", anoFalecimento=" + anoFalecimento +
                '}';
    }
}

interface LivroRepository extends JpaRepository<Livro, Long> {
    List<Livro> findByIdioma(String idioma);
}

interface AutorRepository extends JpaRepository<Autor, Long> {
    List<Autor> findByAnoFalecimentoGreaterThanEqual(int ano);
}

SyntaxError: invalid syntax (<ipython-input-6-828e10e85325>, line 12)